# Semantic Memory

In [10]:
# %pip install -U langmem langgraph

In [11]:
from dotenv import load_dotenv
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

In [12]:
load_dotenv()

True

In [13]:
checkpointer = InMemorySaver()
store = InMemoryStore()

In [14]:
agent = create_react_agent("google_genai:gemini-2.5-pro-preview-05-06", tools=[], store=store, checkpointer=checkpointer)

In [15]:
def chat(agent, txt, thread_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, config={"configurable": {"thread_id": thread_id}})
    return result_state["messages"][-1].content

In [16]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

'Okay, training for a half marathon in 2 months (8 weeks) is ambitious, especially if you\'re starting from a low fitness base. This plan assumes you can currently run at least 2-3 miles (3-5 km) comfortably a few times a week. If not, you might need to adjust your goal to "completing" rather than aiming for a specific time, and be extra cautious about injuries.\n\n**Crucial Considerations Before You Start:**\n\n1.  **Consult Your Doctor:** Especially if you have any pre-existing health conditions.\n2.  **Listen to Your Body:** This is paramount. Rest days are as important as training days. If you feel pain (not just discomfort), take an extra rest day or see a professional.\n3.  **Proper Gear:** Invest in good running shoes fitted for your foot type.\n4.  **Hydration & Nutrition:** Fuel your body well. Drink plenty of water throughout the day and eat a balanced diet. Start practicing with any gels/chews you might use on race day during your long runs.\n5.  **Flexibility:** This is a t

In [17]:
chat(agent, "Nice! Wish me luck!", thread_1)

"Absolutely! **Wishing you the very best of luck with your training and your half marathon!**\n\nYou've got this! Remember to listen to your body, stay consistent, and most importantly, enjoy the journey of pushing your limits and achieving something amazing.\n\nGo crush it! Let me know how it goes! 🎉👟"

In [18]:
thread_2 = "thread-2"
chat(agent, "Nice! Oh thank you! It'll be hard.", thread_2)

"You're very welcome!\n\nAnd yes, I understand that some things can definitely feel that way. But remember, difficult doesn't mean impossible!\n\nIf there's anything I can do to help you break it down, brainstorm, or just provide information as you tackle it, please let me know. Sometimes just talking through the hard parts can make them seem a little less daunting.\n\nYou've got this! 💪"

## Adding Memory

We'll use regular tools.

In [20]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()
agent = create_react_agent("google_genai:gemini-2.5-pro-preview-05-06", tools=memory_tools, store=store, checkpointer=checkpointer)

In [21]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

"Okay, training for a half marathon in 2 months is an ambitious goal, but definitely achievable with a structured plan! Here’s a sample 8-week daily training plan.\n\n**Important Considerations Before You Start:**\n*   **Consult your doctor:** Especially if you're new to running or have any health concerns.\n*   **Listen to your body:** This is crucial. Rest when you need to, and don't push through pain. It's better to take an extra rest day than to get injured.\n*   **Current fitness:** If you're not currently running, you might need to start with shorter distances or incorporate run/walk intervals (e.g., run 2 mins, walk 1 min, repeat), especially in the first few weeks. This plan assumes some basic level of fitness.\n*   **Warm-up & Cool-down:** Always include 5-10 minutes of light cardio (like walking or slow jogging) and dynamic stretches before each run. Cool down with 5-10 minutes of walking and static stretches.\n*   **Pace:**\n    *   **Easy Pace:** You should be able to hold 

In [22]:
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1))

Good luck with your training and the half marathon! I've saved the detailed 8-week plan for you. You can ask me to recall it whenever you need.

Remember to listen to your body, stay consistent, and enjoy the process! You've got this!


In [23]:
thread_2 = "thread-2"
chat(agent, "Remember what I'm supposed to do for my training this week? It's week 3...", thread_2)

"Okay, for week 3 of your half marathon training plan, here's what's on the schedule:\n\n*   **Monday:** Rest\n*   **Tuesday:** Run 3 miles (30-35 min) easy.\n*   **Wednesday:** Cross-Training (XT) 40 min OR Run 2 miles easy + 4x100m strides.\n*   **Thursday:** Run 3 miles (30-35 min) easy.\n*   **Friday:** Rest\n*   **Saturday:** Long Run: 5 miles easy.\n*   **Sunday:** Rest."

In [24]:
chat(agent, "That may be tricky. I just sprained my ankle. Could you update my plan to include more cross training? Be sure to update the existing key of our plan", thread_2)

"I've updated your training plan to focus on non-impact cross-training due to your sprained ankle. I've also added important notes about consulting with your doctor or physical therapist.\n\n**Crucially, you should NOT run until you are cleared by a healthcare professional.**\n\nHere's a suggested modification for the next few weeks, focusing on healing and maintaining fitness:\n\n**Modified Plan (Focus on Ankle Recovery):**\n\n*   **General Approach:** All running activities from the original plan should be replaced with non-impact cross-training (e.g., swimming, cycling if it doesn't cause pain, upper body workouts). Listen to your body, and stop if you feel any pain. The durations below are guides for your cross-training.\n\n*   **Weeks 1-3 (Initial Injury Phase - assuming your injury happened around the start of Week 3):**\n    *   **Focus:** Healing and maintaining fitness with non-impact XT. All activities must be pain-free.\n    *   **Monday:** Rest (elevate and ice your ankle a

## Different User Memories

In [35]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories", "{user_id}")
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

agent = create_react_agent("google_genai:gemini-2.5-pro-preview-05-06", tools=memory_tools, store=store, checkpointer=checkpointer)

In [36]:
def chat(agent, txt, thread_id, user_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, 
                                config={"configurable": {"thread_id": thread_id, "user_id": user_id}})
    return result_state["messages"][-1].content

In [27]:
thread_1 = "thread-1"
user_id = "User-A"
chat(agent, 
     "Hi I'm Will, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare and help me stay honest??",
     thread_1,
     user_id)

"Hi Will! That's a great goal. I can definitely help you with a training plan and keeping you on track.\n\nTo start, could you tell me a bit about your current running experience? For example, how many times a week do you currently run, and what's your average distance? Knowing this will help me tailor a plan that's right for you."

In [28]:
thread_2 = "thread-2"
user_id2 = "User-B"
chat(agent, 
     "Hi I'm John, I'm learning chess - could you help me become great??",
     thread_2,
     user_id2)

"Hi John! That's a great goal. I can definitely help you on your journey to becoming a great chess player. I've just created a memory of your goal.\n\nTo start, what do you find most challenging about chess right now? Knowing that will help me tailor my advice. We can also explore openings, middle-game strategy, endgames, or even analyze famous games."

In [29]:
chat(agent, 
     "Do you remember me liking any sports?",
     thread_1,
     user_id2)

"I don't have a specific memory of you mentioning sports you like. My apologies!\n\nHowever, I do remember that you're currently training for a half marathon, which is a fantastic athletic goal!\n\nHave you told me about other sports you enjoy before? If so, I'd be happy to refresh my memory."

In [30]:
items = store.search(("agent_memories",))
for item in items:
    print(item.namespace, item.value)

('agent_memories', 'User-A') {'content': 'User Will is training for a half marathon to be completed in 2 months. He has requested a daily training plan and assistance in staying honest with his training.'}
('agent_memories', 'User-B') {'content': 'User John is learning chess and aims to become a great player.'}


## "Eager" memory retrieval

We can fetch memories before the first LLM call to simplify its response. Otherwise, it has known and unknown unknowns so will almost always try to search for some subclass of questions.

In [41]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.config import get_store

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

def prompt(state):
    # Search over memories based on the messages
    store = get_store()
    items = store.search(namespace, query=state["messages"][-1].content)
    memories = "\n\n".join(str(item) for item in items)
    system_msg = {"role": "system", "content": f"## Memories:\n\n{memories}"}
    return [system_msg] + state["messages"]
    
agent = create_react_agent("google_genai:gemini-2.5-pro-preview-05-06", prompt=prompt, tools=memory_tools, store=store, checkpointer=checkpointer)

In [42]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a base daily training plan to prepare? Just provide a draft one", thread_1, None)

"Okay, I can help you with that! Here's a sample base daily training plan for a half marathon. Remember, this is a general guide, and you may need to adjust it based on your current fitness level and how your body responds. It's also a good idea to consult with a doctor or a running coach.\n\n**Key:**\n\n*   **Easy Run:** A comfortable pace where you can hold a conversation.\n*   **Tempo Run:** A comfortably hard pace, sustainable for a longer period.\n*   **Interval Training:** Alternating between high-intensity bursts and recovery periods.\n*   **Long Run:** A slow, steady run designed to build endurance.\n*   **Cross-Training (XT):** Activities like swimming, cycling, or strength training.\n*   **Rest:** Essential for recovery and injury prevention.\n\n**Weekly Schedule (repeat for 8 weeks, gradually increasing mileage):**\n\n*   **Monday:** Rest or XT (30-45 minutes)\n*   **Tuesday:** Easy Run (3-5 miles)\n*   **Wednesday:** Tempo Run (3-4 miles, with 1-2 miles at tempo pace) or In

In [43]:
print(chat(agent, "Nice! Wish me luck! Please note down this detailed plan in memories for me :)", thread_1, None))

Good luck with your training! I've saved this detailed plan in my memory for you. Feel free to ask me any questions about it as you go, or if you need any modifications. You've got this!


In [44]:
thread_2 = "thread-2"
chat(agent, "What I'm supposed to do for my training this week? It's week 3...", thread_2, None)

"Okay, for Week 3 of your half marathon training, here’s what your schedule looks like:\n\n*   **Monday:** Rest or Cross-Training (XT) for 30-45 minutes.\n*   **Tuesday:** Easy Run for 3-5 miles.\n*   **Wednesday:** Tempo Run (3-4 miles, with 1-2 miles at tempo pace) or Interval Training (for example, 6x400m with recovery jogs in between).\n*   **Thursday:** Easy Run for 3-5 miles.\n*   **Friday:** Rest or Cross-Training (XT) for 30-45 minutes.\n*   **Saturday:** Long Run. This week, you should aim for 5-7 miles, increasing from last week's distance.\n*   **Sunday:** Easy Run for 2-3 miles, or Rest.\n\nRemember to include a warm-up before each run and a cool-down afterward. Listen to your body, stay hydrated, and make sure you're fueling properly!"